In [50]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import csv
import pandas as pd

BASE = 'https://www.schwabjobs.com'

 
def get_category(response):
    bs_obj = BeautifulSoup(response, "html.parser")
    if bs_obj:
        cat_table = bs_obj.find_all("table",{"class": "list-categores-table"})

        catLinks = []
        numPages = []

        for page in cat_table:
            linkTags = page.find_all('a', attrs={'href': re.compile("/ListJobs")})
            divTags = page.find_all('div')
            for tag in linkTags:
                catLinks.append(tag.get('href'))
            for div in divTags:
                num = int(re.search(r'\d+', div.text).group())
                numPages.append(num)
        return dict(zip(catLinks, numPages))
        
def get_pages(categories):
    pageLinks = []
    for key, value in categories.items():
        if("---" not in key):
            if(value > 30):
                pageCount = 1
                for i in range(1,value,29):
                    pageLinks.append(BASE + key + "Page-" + str(pageCount))
                    pageCount = pageCount + 1
                
            else:
                pageLinks.append(BASE + key)
    return pageLinks
        

def scrape_pages(pages):
    for cat in categories:
        cat_page = urlopen(BASE + cat)
        soup = BeautifulSoup(cat_page)
        table = soup.find('table', 'JobListTable')
        links = []
        for link in table.findAll('a', attrs={'href': re.compile("/ShowJob")}):
            page = BASE + link.get('href')
            print(scrape(page))
            break
        break
        
def scrape(page):
    
    posting = urlopen(page)
    soup2 =  BeautifulSoup(posting)

    value = soup2.findAll("div", {"class": "desc"})
    
    headers = ["Job No", "Institution", "List Id {1,2,3}", "URL (URL of the job posting)", "List id (1,2,3)"]
    values = [1, "Charles Schwab", 1, page, 1]
    
    indices = list(range(1, 101))
    
    wordCountDict = dict.fromkeys(indices,0)
    postingDict = dict(zip(headers, values))
    
    enc = 'utf-8'
    with open('freq100.csv', 'r', encoding = enc) as f:
        reader = csv.reader(f)
        keywords = list(reader)
    arr = [i[0] for i in keywords]

    for val in value:
        st = val.text.split()
        for word in st:
            if(word in arr):
                ind = arr.index(word)
                wordCountDict[ind] = wordCountDict.get(ind, 0) + 1
    dataDict = postingDict.copy()  
    return {**postingDict, **wordCountDict} 


                
if __name__ == '__main__':
    res = urlopen("https://www.schwabjobs.com/List/Custom/Schwab-Job-Category")
    categories = get_category(res)
    pages = get_pages(categories)
    scrape_pages(pages)

{'Job No': 1, 'Institution': 'Charles Schwab', 'List Id {1,2,3}': 1, 'URL (URL of the job posting)': 'https://www.schwabjobs.com/ShowJob/Id/1939618/Deal-Desk-Sourcing-Manager/', 'List id (1,2,3)': 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 3, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 5, 57: 0, 58: 0, 59: 4, 60: 1, 61: 0, 62: 0, 63: 1, 64: 0, 65: 0, 66: 0, 67: 0, 68: 0, 69: 0, 70: 0, 71: 0, 72: 0, 73: 0, 74: 1, 75: 0, 76: 0, 77: 0, 78: 0, 79: 0, 80: 0, 81: 0, 82: 0, 83: 0, 84: 0, 85: 0, 86: 0, 87: 0, 88: 0, 89: 0, 90: 3, 91: 0, 92: 0, 93: 0, 94: 0, 95: 1, 96: 0, 97: 0, 98: 0, 99: 0, 100: 0}
